# Schnorr digital signatures and verification
## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes the point $P = dG = (P_x, P_y)$: his public key is the $x$-coordinate, $P_x$

In [1]:
def hashPDF(file, BLOCK_SIZE):
    # hash=sha256()
    with open(file, 'rb') as f: # Open the file to read it's bytes
        fb = f.read(BLOCK_SIZE) # Read from the file. Take in the amount declared above
        M = sl.sha256(fb)
    return M
file = "D:/Documentos/pdf-test.pdf"
size = os.path.getsize(file) 

In [28]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))

ckp.create_keypair(n_keys)["users"]

[{'nameUser': '0',
  'privateKey': '492d111b2984a7f6573f52d143feb86738f778dfdd9fa3da6875a033fe20d30d',
  'publicKey': 'e1c4ca7556451f55a801e896aec3228274427bceef744b4a437e3e67b8273a09'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, the $x$-coordinate of $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A computes $e = h(R || P || M)$
- A computes $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [4]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =", user[0]["publicKey"])
print("Signature =",sig.hex())

PublicKey = 85b8f9f70afaedf6e102295e88b4295bad954e7f7e9d774a8a8e050570b22bb2
Signature = 691c21c137f7dc77f837c84725edf3a5982a21d6f2b832aa12caac5485e526d0e1df0b501dec73e212645f2e65b3d1bdacec83f8d08016b3ddf34daf71f8021f


## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user $i$ computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(R || \tilde{X}|| M)$
- Every user computes $s_i = r_i + cd_i a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

In [3]:
import create_keypair as ckp
import schnorr_lib as sl

users = [{'nameUser': '0', 'privateKey': '2fa3126813a243984d29bfd7c76d46a1fa7747219b993b940d26095b42699f49','publicKey': '4390c599fb59aac410a218e5bde50a1edbffba61cfe0b3ca8dd7b7f047b7f8db'},
 {'nameUser': '1',
            'privateKey': '428edc2fb282ead8a19107a8f3f3df1a72cae3842f9fea8cd8ec053d8b4704b3',
            'publicKey': '9b03a64a6065cca09e3babe573db1108439ef49d6a64d346cc619cb9abf4a642'},
 {'nameUser': '2',
            'privateKey': 'f7f9406a6c092ae4f566874256e07a80841e66304c0318cebd555b3260f8ff1a',
            'publicKey': 'ab30f2e76ebf786eb44b0cfed63756a2ea116c96f355409e2974afd1305085dc'}]

#n_keys = int(input("Insert the number of keys to be generated:"))
#users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X,l = sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig)



#print('The private keys are: ')
#print('--------------------------')
#for u in users:
#    pk = u['privateKey']
#    print('Usuario ' + u['nameUser'] + ':', pk)

c 87466471819044610412310485724871971209301682117051771801384901549329348931851
ki 85739377958203278443205717736587277550532327825568154362338936696224811329705
ki 73520801332297077507381683831385379556539305269055373205715619645751287071659
ki 19292908017424699450613258118231952273741240934008904546857962223447842447481
Suma 110758112968701878639947763462830229395839766979738652564084431787877783796057
sig b"\xb6\x12z\x12*J\xb0\x13\x85\xa7\xbb\x8b1I]O\x99\xdcv' \x85'\xf2e\x17\x05\xca\x11\xd7\x0cr\xf4\xde\xde\xd2\x9e\xd3\\id\xa2+\xd0\x82e\xcb0{M\xf7\x85\x7f]\x8a?fm$$\x82\xefyY"
Aggregated key = 6f206c6fbca22baddb2bf13d5b8d32e704683a8e7fb76d8efaa5eeb9d92f67d0
Signature = b"\xb6\x12z\x12*J\xb0\x13\x85\xa7\xbb\x8b1I]O\x99\xdcv' \x85'\xf2e\x17\x05\xca\x11\xd7\x0cr\xf4\xde\xde\xd2\x9e\xd3\\id\xa2+\xd0\x82e\xcb0{M\xf7\x85\x7f]\x8a?fm$$\x82\xefyY"


## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(R_1||...||R_{\nu}|| \tilde{X} || M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(R || \tilde{X}||M)$, then every user $i$ computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

In [31]:
import create_keypair as ckp
import schnorr_lib as sl
import os
# from hashlib import sha256

n_keys = int(input("Insert the number of keys to be generated:"))
# n_keys = 2
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = 3dd9f89da17633fa5be72986a605afdc3477a2663bd0661198d59a93d0fb3f62
Signature = ca50387b9e098aac7f7396551091f834c0a0c760e40b5fcedb598a3de2fc4136e871d779bfc01300e8806f26779628070fccf65b23fd58595f7deb11f89d6d00


## Verify Schnorr

- $B$, the receiver, computes the point $sG$
- $B$ computes $e = h(R || P || M)$ $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$
- if $sG = R + eP$, the signature is valid  $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$

In [3]:
import schnorr_lib as sl

# M = input("Insert the message to verify:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is VALID for this message and this public key


In [19]:
e = 3216008939048214360597857449023540673426933993762071189199738912853539483825
e = sl.bytes_from_int(e)
e.hex()

'071c32515fb38fec60e31b2c9213c41a6418528553c47790e1ee0d763d0cf0b1'

In [24]:
priv = '2fa3126813a243984d29bfd7c76d46a1fa7747219b993b940d26095b42699f49'
di = sl.int_from_hex(priv)
Pi = sl.pubkey_point_gen_from_int(di)
Pi

(30560750627779365274982286354606316786288232992182288084301975318891836471515,
 34909107037968473317583628963619474172938085703994472818439190251176098246240)

In [22]:
import json
import ast
import schnorr_lib as sl

# Parámetros de las Curvas Elípticas
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
G = (0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
     0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8)

with open('claves.json') as json_file:
        data = json.load(json_file)

L = b''
for i in data: 
    Pi = ast.literal_eval(i['Clave Publica'])
    byte_clave = sl.bytes_from_point(Pi)
    L += byte_clave

L = sl.sha256(L)

Rsum = None
X = None

lista_ai = []
lista_ki = []
for u in data:
    Pi = ast.literal_eval(u['Clave Publica'])
    #    byte_clave = bytes_from_int(int_clave)
    int_ri = ast.literal_eval(u['Ri'])
    ki = u['Ki']

    ai = sl.int_from_bytes(sl.sha256(L + sl.bytes_from_point(Pi)))
    lista_ai.append(ai)

    X = sl.point_add(X, sl.point_mul(Pi, ai))

    Rsum = sl.point_add(Rsum, int_ri)   
    lista_ki.append(ki)

X = sl.bytes_from_point(X)

if not sl.has_even_y(X):
    cont = 0
    for i in lista_ai:
        lista_ai[cont] = n- i
        cont +=1 

if not sl.has_even_y(Rsum):
    cont = 0
    for i in lista_ki:
        lista_ki[cont] = n- i
        cont +=1 

print("X", X.hex())

c = sl.int_from_bytes(sl.tagged_hash("BIP0340/challenge", (sl.bytes_from_point(Rsum) + sl.bytes_from_point(X) + M))) % n

print("c",c)

with open('users.json', 'r') as f:
    data_1 = json.load(f)

priv_key = data_1['users'][0]['privateKey']
priv_key = sl.int_from_hex(priv_key)

ki = int(data[0]['Ki'])

s = (c * priv_key* lista_ai[0]+ ki) % n

print("s",s)


X 6f206c6fbca22baddb2bf13d5b8d32e704683a8e7fb76d8efaa5eeb9d92f67d0
c 45050382748405608257714015408862828451509166462202937335678800998190154071639
s 102309678818734876725165232509441581969069320118945076837614101687563476087585


In [20]:
import ast
di = sl.int_from_hex(users[0]['privateKey'])
Pi = str(sl.pubkey_point_gen_from_int(di))

p = ast.literal_eval(Pi)
o = lista_ai[0]
print(p)
print(o)

(30560750627779365274982286354606316786288232992182288084301975318891836471515, 34909107037968473317583628963619474172938085703994472818439190251176098246240)
13682049056019787302705909686695135530716896051601928761602000182282948405842


In [21]:
X = None
sl.point_add(X, sl.point_mul(p, o))


(93046424413264835156364268111996672690167960588753920482589198875457756262099,
 9431756804066275983505997047088300076029467773329466319897419981858038319810)

In [6]:
import json
with open('firmas_esquema.json', 'r') as f:
    data = json.load(f)

Rsum = data[0]['Sumatoria R']

s_suma = 0
for i in data: 
    s_i = i['firma']
    s_suma += s_i

s_suma = s_suma % n

signature_bytes = sl.bytes_from_point(Rsum) + sl.bytes_from_int(s_suma)


X = sl.bytes_from_point(data[0]['firma agregada'])


#        signature_bytes = sl.int_from_bytes(signature_bytes)

X = sl.int_from_bytes(X)

signature_bytes = str(signature_bytes)

NameError: name 'n' is not defined

In [21]:
len("56323776146680878002305240761724537382079791310488645351282546751374155493451")

77

In [9]:

if len(M) != 32:
    raise ValueError('The message must be a 32-byte array.')

with open('firma_digital.json') as json_file:
    data = json.load(json_file)

clave_p = data['Firma']
sig = ast.literal_eval(clave_p)

clave_agregada = int(data['Firma Agregada'])

pubkey = sl.bytes_from_int(clave_agregada)

    
if len(pubkey) != 32:
    raise ValueError('The public key must be a 32-byte array.')
if len(sig) != 64:
    raise ValueError('The signature must be a 64-byte array.')
P = sl.lift_x_even_y(pubkey)
r = sl.get_int_R_from_sig(sig)
s = sl.get_int_s_from_sig(sig)

if (P is None) or (r >= p) or (s >= n):
    print(False)
e = sl.int_from_bytes(sl.tagged_hash("BIP0340/challenge", sl.get_bytes_R_from_sig(sig) + pubkey + M)) % n
R = sl.point_add(sl.point_mul(G, s), sl.point_mul(P, n - e))
#if (R is None) or (not sl.has_even_y(R)):
#    print("Please, recompute the sign. R is None or has even y")
#    print(False)
if sl.x(R) != r:
    print("There's something wrong")
    print(False)
print(True)

True


In [19]:
cont = 0
for i in lista_ai:
    lista_ai[cont] = n- i
    cont +=1 

In [20]:
lista_ai

[102110040181296408120865075321992772322120668227472975621003162959235213088495,
 16749993410327024776330253745095961615675833494624748701588071676182647905199,
 40125695165738213180965383003819534378471557564987713598383313323946091125992]

In [17]:
lista_ki

[71383288774214860738898071301071077361478640860560947131238718176753047698122,
 80507694412587728907085711536093080521859837694789296503355957287480689915560,
 60383146465786567092195608159110189617627632460229778036902891045464467115420]

In [3]:
files=['json_1.json','json_2.json','json_3.json']

In [4]:
import schnorr_lib as sl
sl.merge_JsonFiles(files)